In [90]:
!pip install autogluon

In [91]:
import pandas as pd
import numpy as np
import os
import random
from tqdm.auto import tqdm
# import optuna

DATA_PATH = '/kaggle/input/jeju-dataset/'

SEED = 42

In [92]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(SEED) # Seed 고정

In [93]:
import numpy as snp
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings(action='ignore')

In [94]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn import tree
# from catboost import CatBoostRegressor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

from sklearn.model_selection import KFold, train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, make_scorer

```
item : 품목 코드
TG : 감귤
BC : 브로콜리
RD : 무
CR : 당근
CB : 양배추
corporation : 유통 법인 코드
법인 A부터 F 존재
location : 지역 코드
J : 제주도 제주시
S : 제주도 서귀포시
supply(kg) : 유통된 물량, kg 단위
price(원/kg) : 유통된 품목들의 kg 마다의 가격, 원 단위
```

# DATA Load

In [95]:
train_df = pd.read_csv(f"{DATA_PATH}train.csv")
test_df = pd.read_csv(f"{DATA_PATH}test.csv")
international = pd.read_csv(f"{DATA_PATH}international_trade.csv")
submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")

In [96]:
train_df.head()

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0


In [97]:
train_df.rename(columns={'supply(kg)':'supply', 'price(원/kg)':'price'}, inplace=True)
train_df

,ID,timestamp,item,corporation,location,supply,price
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0
...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0


In [98]:
international.head()

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지
0,2019-01,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990
1,2019-01,양파,821330,222,4003206,1118,-896
2,2019-01,쪽파,60,1,93405,128,-127
3,2019-01,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562
4,2019-01,방울다다기 양배추,0,0,7580,38,-38


# Preprocessing

In [99]:
# 공휴일
holi_weekday = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01']

In [100]:
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
train_df['year'] = train_df['timestamp'].dt.year
train_df['month'] = train_df['timestamp'].dt.month
train_df['week'] = train_df['timestamp'].dt.isocalendar().week
train_df['day'] = train_df['timestamp'].dt.day
train_df['day_of_week'] = train_df['timestamp'].dt.dayofweek
train_df['sin_week'] = np.sin(2 * np.pi * train_df['week'] / 4)
train_df['cos_week'] = np.cos(2 * np.pi * train_df['week'] / 4)
train_df['sin_dayofweek'] = np.sin(2 * np.pi * (train_df['day_of_week'] + 1) / 7)
train_df['cos_dayofweek'] = np.cos(2 * np.pi * (train_df['day_of_week'] + 1) / 7)
train_df['holiday'] = np.where((train_df.day_of_week >= 5) | (train_df.timestamp.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0)

test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])
test_df['year'] = test_df['timestamp'].dt.year
test_df['month'] = test_df['timestamp'].dt.month
test_df['week'] = test_df['timestamp'].dt.isocalendar().week
test_df['day'] = test_df['timestamp'].dt.day
test_df['day_of_week'] = test_df['timestamp'].dt.dayofweek
test_df['sin_week'] = np.sin(2 * np.pi * test_df['week'] / 4)
test_df['cos_week'] = np.cos(2 * np.pi * test_df['week'] / 4)
test_df['sin_dayofweek'] = np.sin(2 * np.pi * (test_df['day_of_week'] + 1) / 7)
test_df['cos_dayofweek'] = np.cos(2 * np.pi * (test_df['day_of_week'] + 1) / 7)
test_df['holiday'] = np.where((test_df.day_of_week >= 5) | (test_df.timestamp.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0)

## 월별 품목의 가격 추가(mean, max)

In [101]:
train_df['month_of_year'] = train_df['timestamp'].dt.to_period('M')
test_df['month_of_year'] = test_df['timestamp'].dt.to_period('M')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,day,day_of_week,sin_week,cos_week,sin_dayofweek,cos_dayofweek,holiday,month_of_year
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,1.0,0.0,0.974928,-0.222521,1,2019-01
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,2,2,1.0,0.0,0.433884,-0.900969,0,2019-01
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,3,3,1.0,0.0,-0.433884,-0.900969,0,2019-01
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,4,4,1.0,0.0,-0.974928,-0.222521,0,2019-01
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,5,5,1.0,0.0,-0.781831,0.623490,1,2019-01


In [102]:
item_month_price = train_df.groupby(['item', 'month_of_year'])
agg = {'price' : ['mean', 'max']}
price_df = item_month_price.agg(agg).reset_index()
price_df

item month_of_year        price        
                               mean     max
0     BC       2019-01  1278.086022  2689.0
1     BC       2019-02  1152.563492  2925.0
2     BC       2019-03  1772.620072  4149.0
3     BC       2019-04  2288.400000  6929.0
4     BC       2019-05   658.265233  5965.0
..   ...           ...          ...     ...
250   TG       2022-11  1361.850000  2526.0
251   TG       2022-12  1696.103226  4202.0
252   TG       2023-01  1782.693548  8000.0
253   TG       2023-02  2250.250000  8500.0
254   TG       2023-03  3442.633333  6382.0

[255 rows x 4 columns]

In [103]:
price_df.columns = ['item', 'month_of_year', 'month_price_mean', 'month_price_max']
train_df = pd.merge(train_df, price_df, on=['item', 'month_of_year'], how='left')
train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,day,day_of_week,sin_week,cos_week,sin_dayofweek,cos_dayofweek,holiday,month_of_year,month_price_mean,month_price_max
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,1.0,0.0,0.974928,-0.222521,1,2019-01,1485.383871,4031.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,2,2,1.0,0.0,0.433884,-0.900969,0,2019-01,1485.383871,4031.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,3,3,1.0,0.0,-0.433884,-0.900969,0,2019-01,1485.383871,4031.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,4,4,1.0,0.0,-0.974928,-0.222521,0,2019-01,1485.383871,4031.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,5,5,1.0,0.0,-0.781831,0.623490,1,2019-01,1485.383871,4031.0


In [104]:
# 테스트셋 반영
test_df = pd.merge(test_df, price_df, on=['item', 'month_of_year'], how='left')
test_df.head()

,ID,timestamp,item,corporation,location,year,month,week,day,day_of_week,sin_week,cos_week,sin_dayofweek,cos_dayofweek,holiday,month_of_year,month_price_mean,month_price_max
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,9,4,5,1.0,0.0,-7.818315e-01,0.623490,1,2023-03,3442.633333,6382.0
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,9,5,6,1.0,0.0,-2.449294e-16,1.000000,1,2023-03,3442.633333,6382.0
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,10,6,0,0.0,-1.0,7.818315e-01,0.623490,0,2023-03,3442.633333,6382.0
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,10,7,1,0.0,-1.0,9.749279e-01,-0.222521,0,2023-03,3442.633333,6382.0
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,10,8,2,0.0,-1.0,4.338837e-01,-0.900969,0,2023-03,3442.633333,6382.0


## 월별 품목의 물량 추가(mean, max)

In [105]:
item_month_supply = train_df.groupby(['item', 'month_of_year'])
agg = {'supply' : ['mean', 'max']}
supply_df = item_month_supply.agg(agg).reset_index()
supply_df

item month_of_year        supply          
                                mean       max
0     BC       2019-01   2494.161290   14584.0
1     BC       2019-02   2288.412698   15184.0
2     BC       2019-03   2056.946237   14048.0
3     BC       2019-04   1347.229630   10136.0
4     BC       2019-05    124.301075    3792.0
..   ...           ...           ...       ...
250   TG       2022-11  29303.189000  104781.0
251   TG       2022-12  34294.561290  129891.0
252   TG       2023-01  27517.410968  127048.0
253   TG       2023-02  19588.908571   78797.0
254   TG       2023-03  16526.743333   47819.0

[255 rows x 4 columns]

In [106]:
supply_df.columns = ['item', 'month_of_year', 'month_supply_mean', 'month_supply_max']
train_df = pd.merge(train_df, supply_df, on=['item', 'month_of_year'], how='left')
train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,sin_week,cos_week,sin_dayofweek,cos_dayofweek,holiday,month_of_year,month_price_mean,month_price_max,month_supply_mean,month_supply_max
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,1.0,0.0,0.974928,-0.222521,1,2019-01,1485.383871,4031.0,47433.957097,268073.4
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,1.0,0.0,0.433884,-0.900969,0,2019-01,1485.383871,4031.0,47433.957097,268073.4
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,1.0,0.0,-0.433884,-0.900969,0,2019-01,1485.383871,4031.0,47433.957097,268073.4
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,1.0,0.0,-0.974928,-0.222521,0,2019-01,1485.383871,4031.0,47433.957097,268073.4
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,1.0,0.0,-0.781831,0.623490,1,2019-01,1485.383871,4031.0,47433.957097,268073.4


In [107]:
# 테스트셋 반영
test_df = pd.merge(test_df, supply_df, on=['item', 'month_of_year'], how='left')
test_df.head()

,ID,timestamp,item,corporation,location,year,month,week,day,day_of_week,sin_week,cos_week,sin_dayofweek,cos_dayofweek,holiday,month_of_year,month_price_mean,month_price_max,month_supply_mean,month_supply_max
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,9,4,5,1.0,0.0,-7.818315e-01,0.623490,1,2023-03,3442.633333,6382.0,16526.743333,47819.0
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,9,5,6,1.0,0.0,-2.449294e-16,1.000000,1,2023-03,3442.633333,6382.0,16526.743333,47819.0
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,10,6,0,0.0,-1.0,7.818315e-01,0.623490,0,2023-03,3442.633333,6382.0,16526.743333,47819.0
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,10,7,1,0.0,-1.0,9.749279e-01,-0.222521,0,2023-03,3442.633333,6382.0,16526.743333,47819.0
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,10,8,2,0.0,-1.0,4.338837e-01,-0.900969,0,2023-03,3442.633333,6382.0,16526.743333,47819.0


In [108]:
# 공급과 가격의 상관관계
sup_price_corr = train_df['supply'].corr(train_df['price'])
sup_price_corr

-0.0022967366004861545

## 유닉스 타임스탬프
- 1970년 1월 1일부터의 초 단위 경과 시간으로 변환

In [109]:
# # timestamp
# train_df['timestamp'] = (train_df['timestamp'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')
# test_df['timestamp'] = (test_df['timestamp'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')

# train_df['timestamp']

In [110]:
# # month
# train_df['month'] = (train_df['month'].dt.to_timestamp() - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')
# test_df['month'] = (test_df['month'].dt.to_timestamp() - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')

# test_df['month']

## international 데이터 merge

In [111]:
international = international.rename(columns={
    '기간': 'period',
    '품목명': 'item',
    '수출 중량': 'exWeight',
    '수출 금액': 'exValue',
    '수입 중량': 'imWeight',
    '수입 금액': 'imValue',
    '무역수지': 'tradeBalance'
})

international.head()

,period,item,exWeight,exValue,imWeight,imValue,tradeBalance
0,2019-01,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990
1,2019-01,양파,821330,222,4003206,1118,-896
2,2019-01,쪽파,60,1,93405,128,-127
3,2019-01,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562
4,2019-01,방울다다기 양배추,0,0,7580,38,-38


In [112]:
# international['month'] = pd.to_datetime(international['month'], format='%Y-%m')
# international['month'] = (international['month'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')

In [113]:
international['year'] = international['period'].apply(lambda x : int(x[0:4]))
international['month'] = international['period'].apply(lambda x : int(x[5:7]))
international.drop(columns = 'period', inplace=True)
international

,item,exWeight,exValue,imWeight,imValue,tradeBalance,year,month
0,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990,2019,1
1,양파,821330,222,4003206,1118,-896,2019,1
2,쪽파,60,1,93405,128,-127,2019,1
3,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562,2019,1
4,방울다다기 양배추,0,0,7580,38,-38,2019,1
...,...,...,...,...,...,...,...,...
1269,포포(papaw)[파파야(papaya)],0,0,23830,71,-71,2023,2
1270,사과,135165,351,0,0,351,2023,2
1271,배,2206012,5411,1,0,5411,2023,2
1272,신 체리[프루너스 체라서스(Prunus cerasus)],5,0,0,0,0,2023,2


In [114]:
selected_items = international[international['item'].str.contains('감귤|브로콜리|무|당근|양배추')].copy()

selected_items['item'] = selected_items['item'].replace({
    '감귤': 'TG',
    '브로콜리': 'BC',
    '무': 'RD',
    '당근': 'CR',
    '양배추': 'CB'
}, regex = True)

selected_items['item'] = selected_items['item'].str.replace('.*감귤.*', 'TG')
selected_items['item'] = selected_items['item'].str.replace('.*브로콜리.*', 'BC')
selected_items['item'] = selected_items['item'].str.replace('.*무.*', 'RD')
selected_items['item'] = selected_items['item'].str.replace('.*당근.*', 'CR')
selected_items['item'] = selected_items['item'].str.replace('.*양배추.*', 'CB')

selected_items

,item,exWeight,exValue,imWeight,imValue,tradeBalance,year,month
3,꽃CB와 BC(broccoli),160,1,638913,563,-562,2019,1
4,방울다다기 CB,0,0,7580,38,-38,2019,1
5,CB,184650,94,395802,90,4,2019,1
8,CR,23150,22,7466150,2955,-2934,2019,1
12,RD화과,2627,23,94529,464,-441,2019,1
...,...,...,...,...,...,...,...,...
1250,CB,13188,13,377456,104,-91,2023,2
1253,CR,22510,20,9260020,3758,-3737,2023,2
1254,순RD,4000,4,2,0,4,2023,2
1258,RD화과,1319,14,104566,454,-440,2023,2


In [115]:
selected_items.item.value_counts()

꽃CB와 BC(broccoli)    50
방울다다기 CB             50
CB                   50
CR                   50
RD화과                 50
TG                   50
순RD                   6
Name: item, dtype: int64

In [116]:
train_df = train_df.merge(selected_items, on=['year', 'month', 'item'], how='left')
test_df = test_df.merge(selected_items, on=['year', 'month', 'item'], how='left')


train_df['exPrice'] = train_df['exValue'] / train_df['exWeight']
train_df['imPrice'] = train_df['imValue'] / train_df['imWeight']

test_df['exPrice'] = test_df['exValue'] / test_df['exWeight']
test_df['imPrice'] = test_df['imValue'] / test_df['imWeight']

In [117]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

train_df.isnull().sum().sum() , test_df.isnull().sum().sum()

(0, 0)

## 기타 피쳐추가

In [118]:
# 품목별 평균 가격
item_mean_price = train_df.groupby('item')['price'].mean().reset_index()
item_mean_price.columns = ['item', 'item_mean_price']

train_df = train_df.merge(item_mean_price, on='item', how='left')
test_df = test_df.merge(item_mean_price, on='item', how='left')

# 법인별 평균 가격
corp_mean_price = train_df.groupby('corporation')['price'].mean().reset_index()
corp_mean_price.columns = ['corporation', 'corp_mean_price']

train_df = train_df.merge(corp_mean_price, on='corporation', how='left')
test_df = test_df.merge(corp_mean_price, on='corporation', how='left')

# 지역별 평균 가격
location_mean_price = train_df.groupby('location')['price'].mean().reset_index()
location_mean_price.columns = ['location', 'location_mean_price']

train_df = train_df.merge(location_mean_price, on='location', how='left')
test_df = test_df.merge(location_mean_price, on='location', how='left')

# Encoding

## 원핫

In [119]:
# # item OneHotEncoding
# train_df = pd.get_dummies(train_df, columns=['item'])
# test_df = pd.get_dummies(test_df, columns=['item'])

## 라벨

In [120]:
obj_cols = ['item', 'corporation', 'location']

for col in obj_cols:
    enc = LabelEncoder()
    train_df[col]=enc.fit_transform(train_df[col])
    test_df[col]=enc.transform(test_df[col])

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,exWeight,exValue,imWeight,imValue,tradeBalance,exPrice,imPrice,item_mean_price,corp_mean_price,location_mean_price
0,TG_A_J_20190101,2019-01-01,4,0,0,0.0,0.0,2019,1,1,...,58368.0,172.0,0.0,0.0,172.0,0.002947,0.0,3177.442022,1141.704458,1062.35148
1,TG_A_J_20190102,2019-01-02,4,0,0,0.0,0.0,2019,1,1,...,58368.0,172.0,0.0,0.0,172.0,0.002947,0.0,3177.442022,1141.704458,1062.35148
2,TG_A_J_20190103,2019-01-03,4,0,0,60601.0,1728.0,2019,1,1,...,58368.0,172.0,0.0,0.0,172.0,0.002947,0.0,3177.442022,1141.704458,1062.35148
3,TG_A_J_20190104,2019-01-04,4,0,0,25000.0,1408.0,2019,1,1,...,58368.0,172.0,0.0,0.0,172.0,0.002947,0.0,3177.442022,1141.704458,1062.35148
4,TG_A_J_20190105,2019-01-05,4,0,0,32352.0,1250.0,2019,1,1,...,58368.0,172.0,0.0,0.0,172.0,0.002947,0.0,3177.442022,1141.704458,1062.35148


In [121]:
train_df['week'] = train_df['week'].astype(float)
train_df['sin_week'] = train_df['sin_week'].astype(float)
train_df['cos_week'] = train_df['cos_week'].astype(float)

test_df['week'] = test_df['week'].astype(float)
test_df['sin_week'] = test_df['sin_week'].astype(float)
test_df['cos_week'] = test_df['cos_week'].astype(float)

In [122]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59397 entries, 0 to 59396
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   59397 non-null  object        
 1   timestamp            59397 non-null  datetime64[ns]
 2   item                 59397 non-null  int64         
 3   corporation          59397 non-null  int64         
 4   location             59397 non-null  int64         
 5   supply               59397 non-null  float64       
 6   price                59397 non-null  float64       
 7   year                 59397 non-null  int64         
 8   month                59397 non-null  int64         
 9   week                 59397 non-null  float64       
 10  day                  59397 non-null  int64         
 11  day_of_week          59397 non-null  int64         
 12  sin_week             59397 non-null  float64       
 13  cos_week             59397 non-

In [123]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1092 entries, 0 to 1091
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   1092 non-null   object        
 1   timestamp            1092 non-null   datetime64[ns]
 2   item                 1092 non-null   int64         
 3   corporation          1092 non-null   int64         
 4   location             1092 non-null   int64         
 5   year                 1092 non-null   int64         
 6   month                1092 non-null   int64         
 7   week                 1092 non-null   float64       
 8   day                  1092 non-null   int64         
 9   day_of_week          1092 non-null   int64         
 10  sin_week             1092 non-null   float64       
 11  cos_week             1092 non-null   float64       
 12  sin_dayofweek        1092 non-null   float64       
 13  cos_dayofweek        1092 non-nul

# AutoGluon

In [124]:
target = train_df['price']
target

0           0.0
1           0.0
2        1728.0
3        1408.0
4        1250.0
          ...  
59392     468.0
59393     531.0
59394     574.0
59395     523.0
59396     529.0
Name: price, Length: 59397, dtype: float64

In [125]:
train_df['item_id'] = train_df.ID.str[0:6]
train_df

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,exValue,imWeight,imValue,tradeBalance,exPrice,imPrice,item_mean_price,corp_mean_price,location_mean_price,item_id
0,TG_A_J_20190101,2019-01-01,4,0,0,0.0,0.0,2019,1,1.0,...,172.0,0.0,0.0,172.0,0.002947,0.0,3177.442022,1141.704458,1062.35148,TG_A_J
1,TG_A_J_20190102,2019-01-02,4,0,0,0.0,0.0,2019,1,1.0,...,172.0,0.0,0.0,172.0,0.002947,0.0,3177.442022,1141.704458,1062.35148,TG_A_J
2,TG_A_J_20190103,2019-01-03,4,0,0,60601.0,1728.0,2019,1,1.0,...,172.0,0.0,0.0,172.0,0.002947,0.0,3177.442022,1141.704458,1062.35148,TG_A_J
3,TG_A_J_20190104,2019-01-04,4,0,0,25000.0,1408.0,2019,1,1.0,...,172.0,0.0,0.0,172.0,0.002947,0.0,3177.442022,1141.704458,1062.35148,TG_A_J
4,TG_A_J_20190105,2019-01-05,4,0,0,32352.0,1250.0,2019,1,1.0,...,172.0,0.0,0.0,172.0,0.002947,0.0,3177.442022,1141.704458,1062.35148,TG_A_J
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,3,5,0,452440.0,468.0,2023,2,9.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,187.073539,259.121799,1062.35148,RD_F_J
59393,RD_F_J_20230228,2023-02-28,3,5,0,421980.0,531.0,2023,2,9.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,187.073539,259.121799,1062.35148,RD_F_J
59394,RD_F_J_20230301,2023-03-01,3,5,0,382980.0,574.0,2023,3,9.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,187.073539,259.121799,1062.35148,RD_F_J
59395,RD_F_J_20230302,2023-03-02,3,5,0,477220.0,523.0,2023,3,9.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,187.073539,259.121799,1062.35148,RD_F_J


In [126]:
test_df['item_id'] = test_df.ID.str[0:6]
test_df

,ID,timestamp,item,corporation,location,year,month,week,day,day_of_week,...,exValue,imWeight,imValue,tradeBalance,exPrice,imPrice,item_mean_price,corp_mean_price,location_mean_price,item_id
0,TG_A_J_20230304,2023-03-04,4,0,0,2023,3,9.0,4,5,...,0.0,0.0,0.0,0.0,0.0,0.0,3177.442022,1141.704458,1062.35148,TG_A_J
1,TG_A_J_20230305,2023-03-05,4,0,0,2023,3,9.0,5,6,...,0.0,0.0,0.0,0.0,0.0,0.0,3177.442022,1141.704458,1062.35148,TG_A_J
2,TG_A_J_20230306,2023-03-06,4,0,0,2023,3,10.0,6,0,...,0.0,0.0,0.0,0.0,0.0,0.0,3177.442022,1141.704458,1062.35148,TG_A_J
3,TG_A_J_20230307,2023-03-07,4,0,0,2023,3,10.0,7,1,...,0.0,0.0,0.0,0.0,0.0,0.0,3177.442022,1141.704458,1062.35148,TG_A_J
4,TG_A_J_20230308,2023-03-08,4,0,0,2023,3,10.0,8,2,...,0.0,0.0,0.0,0.0,0.0,0.0,3177.442022,1141.704458,1062.35148,TG_A_J
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,3,5,0,2023,3,13.0,27,0,...,0.0,0.0,0.0,0.0,0.0,0.0,187.073539,259.121799,1062.35148,RD_F_J
1088,RD_F_J_20230328,2023-03-28,3,5,0,2023,3,13.0,28,1,...,0.0,0.0,0.0,0.0,0.0,0.0,187.073539,259.121799,1062.35148,RD_F_J
1089,RD_F_J_20230329,2023-03-29,3,5,0,2023,3,13.0,29,2,...,0.0,0.0,0.0,0.0,0.0,0.0,187.073539,259.121799,1062.35148,RD_F_J
1090,RD_F_J_20230330,2023-03-30,3,5,0,2023,3,13.0,30,3,...,0.0,0.0,0.0,0.0,0.0,0.0,187.073539,259.121799,1062.35148,RD_F_J


In [127]:
train_dropcols = ['ID', 'supply', 'month_of_year']
test_dropcols = ['ID', 'month_of_year']

In [128]:
train_ft = train_df.drop(columns = train_dropcols)
test_ft = test_df.drop(columns = test_dropcols)

In [129]:
train_ft.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59397 entries, 0 to 59396
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   timestamp            59397 non-null  datetime64[ns]
 1   item                 59397 non-null  int64         
 2   corporation          59397 non-null  int64         
 3   location             59397 non-null  int64         
 4   price                59397 non-null  float64       
 5   year                 59397 non-null  int64         
 6   month                59397 non-null  int64         
 7   week                 59397 non-null  float64       
 8   day                  59397 non-null  int64         
 9   day_of_week          59397 non-null  int64         
 10  sin_week             59397 non-null  float64       
 11  cos_week             59397 non-null  float64       
 12  sin_dayofweek        59397 non-null  float64       
 13  cos_dayofweek        59397 non-

In [130]:
# from autogluon.tabular import TabularPredictor
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
data = TimeSeriesDataFrame(train_ft)

In [140]:
# Set your SEED value
SEED = 42

# Specify the models you want to exclude
excluded_models = ['DeepAR']

# Create the TimeSeriesPredictor
predictor = TimeSeriesPredictor(
    label="price",
    eval_metric="RMSE",
)

# Fit the predictor to your data, excluding DeepAR
predictor.fit(
    train_data=data,
    excluded_model_types=excluded_models,
    time_limit=None,  # You can set a time limit if needed
)

No path specified. Models will be saved in: "AutogluonModels/ag-20231112_153619/"
================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': ['DeepAR'],
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'default',
 'num_val_windows': 1,
 'prediction_length': 1,
 'random_seed': None,
 'target': 'price',
 'time_limit': None,
 'verbosity': 2}
Provided training data set with 59397 rows, 39 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
AutoGluon will save models to AutogluonModels/ag-20231112_153619/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'price'
	past covariates

In [131]:
# predictor = TimeSeriesPredictor(
#     prediction_length=28,
#     target="price",
#     eval_metric="RMSE"
# )
# # seed 고정
# predictor.fit(data, random_seed=SEED)

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'default',
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': 42,
 'target': 'price',
 'time_limit': None,
 'verbosity': 2}
Provided training data set with 59397 rows, 39 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
AutoGluon will save models to AutogluonModels/ag-20231112_140925/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'price'
	past covariates:  ['item', 'corporation', 'location', 'year', 'month', 'week', 'day', 'day_of_week', 'si


KeyboardInterrupt



# 예측

In [ ]:
# 모델 재학습
predictor.refit_full()

In [ ]:
pred = predictor.predict(data, random_seed=SEED)
pred

In [ ]:
pred = pred.reset_index()['mean']
pred

In [ ]:
# 마이너스 값 0으로 치환
pred[pred < 0] = 0

# 제출

In [ ]:
submission

In [ ]:
submission['answer'] = pred
submission

In [ ]:
submission.to_csv("submission24_1112.csv", index=False)